# Setup

In [242]:
from google import genai
from google.genai.types import GenerateContentConfig, SafetySetting, HarmBlockThreshold, HarmCategory, EmbedContentConfig
from IPython.display import Markdown, display
import numpy as np
from pypdf import PdfReader
from typing import *
from pydantic import BaseModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import os
import re
import fitz
import pickle
import json

In [130]:
!gcloud auth application-default login --project irn-77178-lab-ed

PROJECT_ID = "irn-77178-lab-ed"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "europe-west1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

MODEL_ID = "gemini-2.0-flash-001"
TEXT_EMBEDDING_MODEL_ID = "text-embedding-005"

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=VVZCsH1n4TWcEbpUg3uQRDs3g0lfcs&access_type=offline&code_challenge=rIiCPUe7Suigeuf6gCtVTynI72qRCh6-Y8-WHg4hQq8&code_challenge_method=S256


Credentials saved to file: [C:\Users\p129811\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "irn-77178-lab-ed" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this project. You might receive a "quota_exceeded" or "API not enabled" 

# Extraction PDF

## 1ERE VERSION

### pip install

In [ ]:
!pip install --upgrade pypdf

  Obtaining dependency information for pdf2image from https://files.pythonhosted.org/packages/62/33/61766ae033518957f877ab246f87ca30a85b778ebaad65b7f74fa7e52988/pdf2image-1.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for pytesseract from https://files.pythonhosted.org/packages/7a/33/8312d7ce74670c9d39a532b2c246a853861120486be9443eebf048043637/pytesseract-0.3.13-py3-none-any.whl.metadata


### travail

In [ ]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [4]:
old_doc = extract_text_from_pdf("ancien_clean.pdf")
new_doc = extract_text_from_pdf("nouveau.pdf")

print(old_doc[:1000])
print("#################")
print(new_doc[:1000])

Gynécologie
Obstétrique
Front matterChez le même éditeur
Dans la même collection
Activité physique et sportive : facteur de santé, par le Collège français des enseignants en médecine et trauma-
tologie du sport et de l'exercice physique (CFEMTSEP), 2019, 96 pages.
Anatomie et cytologie pathologiques, par le Collège français des pathologistes (CoPath), 3 e édition, 2019, 
416 pages.
Chirurgie maxillo-faciale et stomatologie, par le Collège hospitalo-universitaire français de chirurgie maxillofa -
ciale et stomatologie, 5e édition, 2021, 432 pages.
Dermatologie, par le Collège des enseignants en dermatologie de France (CEDEF), 7e édition, 2017, 472 pages.
Endocrinologie, diabétologie et maladies métaboliques, par le Collège des enseignants d'endocrinologie, dia -
bète et maladies métaboliques (CEEDMM), 5e édition, 2021, 568 pages.
Gériatrie, par le Collège national des enseignants de gériatrie (CNEG), 5e édition, 2021, 400 pages.
Gynécologie obstétrique, par le Collège national des gynéc

### saves

In [12]:
with open("saves/ancien.txt", "w") as f:
    f.write(old_doc)
with open("saves/nouveau.txt", "w") as f:
    f.write(new_doc)

## 2EME VERSION

### travail

In [211]:
toc_pages = {'nouveau.pdf': [5, 21], 'ancien.pdf': [6, 22]}
index_pages = {'nouveau.pdf': 725, 'ancien.pdf': 819}
first_lines = {'nouveau.pdf': 8, 'ancien.pdf': 10}

def extract_titles(docname):
    # Open the PDF
    doc = fitz.open(docname)

    # Extract text from pages 6 to 21 (0-based index: pages 5 to 20)
    toc_text = ""
    for page_num in range(toc_pages[docname][0], toc_pages[docname][1]):
        toc_text += doc[page_num].get_text()

    toc_text = toc_text.splitlines()
    toc_text = [line.strip(' . ').split('. .')[0] for line in toc_text if not line.isdigit() and "table des matières" not in line.lower()][first_lines[docname]:]
    i=0
    while i < len(toc_text):
        if len(toc_text[i])<=3:
            curr = toc_text.pop(i).strip()
            toc_text[i] = f"{curr}. {toc_text[i]}"
        i+=1
    
    return toc_text

old_toc_text = extract_titles("ancien.pdf")
new_toc_text = extract_titles("nouveau.pdf")

for title in old_toc_text:
    print(title)

I. Connaissances Gynécologie
1. Item 9 Éthique médicale
Introduction 
Interruption volontaire de grossesse 
I. Pour comprendre
II. Pour réfléchir 
Diagnostic prénatal 
I. Pour comprendre
II. Pour réfléchir 
Interruption médicale de grossesse 
I. Pour comprendre
II. Pour réfléchir 
Diagnostic préimplantatoire 
I. Pour comprendre
II. Pour réfléchir 
Décisions difficiles aux limites de la viabilité
I. Pour comprendre
II. Pour réfléchir 
2. Item 12 Violences sexuelles
I. Pour comprendre : les différents types de violences sexuelles et le consentement 
A. Définitions et législation 
B. Épidémiologie des violences sexuelles en France
C. Particularités des violences sexuelles au sein du couple (violence conjugale)
D. Signalement judiciaire et réquisition judiciaire 
II. Objectifs de la prise en charge d'une victime de violences sexuelles
III. Accueil et information d'une victime
A. Vérifier la notion de plainte
B. Apprécier les conséquences psychologiques initiales
C. Préparer la victime à l'

In [212]:
def get_chunks(docname, toc_text):
    doc = fitz.open(docname)
    pages = {}
    chunks = []
    doc_text = ""
    for page_num in range(1, toc_pages[docname][0]):
        doc_text += doc[page_num].get_text()
        pages[page_num+1] = doc[page_num].get_text()
        if pages[page_num+1].strip()=="": del pages[page_num+1]
    chunks.append(doc_text)

    doc_text = ""
    for page_num in range(toc_pages[docname][1], index_pages[docname]):
        doc_text += doc[page_num].get_text()
        pages[page_num+1] = doc[page_num].get_text()

    x = [doc_text]
    y = [f"'{title}'" for title in toc_text]
    split_titles = [re.escape(c.strip("'")) for c in y]
    pattern = r"|".join(split_titles)

    result = [part for item in x for part in re.split(pattern, item) if part]
    result = [chunk for chunk in result if len(chunk)]

    index_text = ""
    for page_num in range(index_pages[docname], len(doc)):
        index_text += doc[page_num].get_text()
        pages[page_num+1] = doc[page_num].get_text()

    lines = index_text.splitlines()
    index_chunks = []
    section = []

    for line in lines:
        if len(line.strip()) == 1 and line.strip()!="–":
            if section:
                index_chunks.append("\n".join(section).strip())
                section = []
        section.append(line)
    if section:
        index_chunks.append("\n".join(section).strip())

    chunks.extend(result)
    chunks.extend(index_chunks)

    chunks = [chunk for chunk in chunks if len(chunk)>10]

    return chunks, pages

old_chunks, old_pages = get_chunks("ancien.pdf", old_toc_text)
new_chunks, new_pages = get_chunks("nouveau.pdf", new_toc_text)

### saves

In [220]:
with open("saves/chunks/title_chunks/ancien_chunks.pkl", 'wb') as f:
    pickle.dump(old_chunks, f)
with open("saves/chunks/title_chunks/ancien_pages.pkl", 'wb') as f:
    pickle.dump(old_pages, f)
with open("saves/chunks/title_chunks/nouveau_chunks.pkl", 'wb') as f:
    pickle.dump(new_chunks, f)
with open("saves/chunks/title_chunks/nouveau_pages.pkl", 'wb') as f:
    pickle.dump(new_pages, f)

# Splitting en chunks

### pip install

In [8]:
!pip install langchain tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/6f/07/c67ad1724b8e14e2b4c8cca04b15da158733ac60136879131db05dda7c30/tiktoken-0.9.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ---------------------------------------- 10.2/893.9 kB ? eta -:--:--
   --- ----------------------------------- 71.7/893.9 kB 975.2 kB/s eta 0:00:01
   ----------------- ---------------------- 399.4/893.9 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 893.9/893.9 kB 6.3 MB/s eta 0:00:00


### travail

In [ ]:
def split_text_into_chunks(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    chunks = []
    # Une approche simple par caractères, mais vous pouvez améliorer avec un split par paragraphe ou phrase
    words = text.split()
    current_chunk = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 <= chunk_size:
            current_chunk.append(word)
            current_length += len(word) + 1
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:] + [word] # Garder un chevauchement
            current_length = sum(len(w) + 1 for w in current_chunk)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def split_text_langchain(text: str, chunk_size: int = 250, overlap: int = 25) -> List[str]:
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
    )
    return text_splitter.split_text(text)

In [10]:
chunks_old = split_text_langchain(old_doc)
chunks_new = split_text_langchain(new_doc)

print(f"Nombre de chunks dans l'ancien document: {len(chunks_old)}")
print(f"Premier chunk de l'ancien document: {chunks_old[0]}")
print(f"Nombre de chunks dans le nouveau document: {len(chunks_new)}")
print(f"Premier chunk du nouveau document: {chunks_new[0]}")

Nombre de chunks dans l'ancien document: 3295
Premier chunk de l'ancien document: Gynécologie
Obstétrique
Front matterChez le même éditeur
Dans la même collection
Activité physique et sportive : facteur de santé, par le Collège français des enseignants en médecine et trauma-
tologie du sport et de l'exercice physique (CFEMTSEP), 2019, 96 pages.
Anatomie et cytologie pathologiques, par le Collège français des pathologistes (CoPath), 3 e édition, 2019, 
416 pages.
Chirurgie maxillo-faciale et stomatologie, par le Collège hospitalo-universitaire français de chirurgie maxillofa -
ciale et stomatologie, 5e édition, 2021, 432 pages.
Dermatologie, par le Collège des enseignants en dermatologie de France (CEDEF), 7e édition, 2017, 472 pages.
Nombre de chunks dans le nouveau document: 3279
Premier chunk du nouveau document: Gynécologie 
ObstétriqueGynécologie 
Obstétrique
Sous l’égide du Collège National des Gynécologues et Obstétriciens Français 
et du Collège des Enseignants de Gynécologie-Ob

### saves

In [13]:
import pickle

with open("saves/chunks/ancien_chunks.pkl", "wb") as f:
    pickle.dump(chunks_old, f)
with open("saves/chunks/nouveau_chunks.pkl", "wb") as f:
    pickle.dump(chunks_new, f)

# Encoding

### travail

In [223]:
old_embeddings, new_embeddings = [], []
i, j = 0, 0
step = 10
output_dim = 256

while i < len(old_chunks):
    old_embeddings_curr = client.models.embed_content(
        model=TEXT_EMBEDDING_MODEL_ID,
        contents=old_chunks[i:i+step],
        config=EmbedContentConfig(output_dimensionality=output_dim),
    ).embeddings
    i += step
    old_embeddings.extend(old_embeddings_curr)

while j < len(new_chunks):
    new_embeddings_curr = client.models.embed_content(
        model=TEXT_EMBEDDING_MODEL_ID,
        contents=new_chunks[j:j+step],
        config=EmbedContentConfig(output_dimensionality=output_dim),
    ).embeddings
    j += step
    new_embeddings.extend(new_embeddings_curr)

In [224]:
print(old_embeddings[0].values)
print("###########################")
print(new_embeddings[0].values)

[0.04928146302700043, 0.019012311473488808, -0.0002940752892754972, -0.01007375493645668, -0.07253903895616531, 0.08885599672794342, -0.03869495913386345, 0.00048324360977858305, 0.020312773063778877, -0.02565460093319416, -0.08525916188955307, -0.0401296466588974, -0.017923636361956596, -0.08163035660982132, -0.012016664259135723, 0.014024415984749794, 0.0016072394791990519, -0.06242465600371361, -0.019036341458559036, 0.06927932798862457, 0.0384238138794899, -0.08494329452514648, -0.0777360200881958, -0.013922736980021, 0.0010887710377573967, 0.003563837381079793, -0.06006141006946564, 0.0316009558737278, -0.02698611468076706, 0.018218349665403366, 0.02541608363389969, 0.029401609674096107, 0.070810467004776, 0.004134016577154398, 0.03686606138944626, -0.0064809685572981834, 0.06345990300178528, 0.007368690334260464, 0.038142602890729904, 0.04448585957288742, 0.04392315819859505, -0.013949106447398663, -0.023250365629792213, 0.04103844240307808, -0.009640013799071312, 0.0826252102851

### saves

In [225]:
with open("saves/embeddings/title_chunks/ancien_embeddings.pkl", "wb") as f:
    pickle.dump(old_embeddings, f)
with open("saves/embeddings/title_chunks/nouveau_embeddings.pkl", "wb") as f:
    pickle.dump(new_embeddings, f)

# Vectorisation

### travail

In [226]:
np_old_embeddings = np.array([embed.values for embed in old_embeddings])
np_new_embeddings = np.array([embed.values for embed in new_embeddings])

# Créer des index FAISS pour chaque document
dimension_embedding = np_old_embeddings.shape[1]

old_index = faiss.IndexFlatL2(dimension_embedding)
old_index.add(np_old_embeddings)

new_index = faiss.IndexFlatL2(dimension_embedding)
new_index.add(np_new_embeddings)

# Comparaison & identification des différences

In [248]:
def find_differences(chunks_old: List[str], embeddings_old: np.ndarray, index_old: faiss.IndexFlatL2, pages_old: Dict[int, str],
                     chunks_new: List[str], embeddings_new: np.ndarray, index_new: faiss.IndexFlatL2, pages_new: Dict[int, str],
                     similarity_threshold: float = 0.9) -> List[str]:
    differences = []

    # Parcourir les chunks du document 2 et chercher leur correspondance dans le document 1
    for i, chunk_new in enumerate(chunks_new):
        query_embedding = embeddings_new[i].reshape(1, -1)
        distances, indices = index_old.search(query_embedding, k=1) # Chercher le plus proche dans doc1

        closest_distance = distances[0][0]
        closest_chunk_old_index = indices[0][0]

        # Si la distance est grande (faible similarité), ou si le chunk n'est pas "trouvé" (distance > seuil),
        # cela indique une possible différence ou un ajout.
        # Note: L2 distance plus faible = plus similaire.
        # Vous devrez ajuster le seuil en fonction de vos observations.
        if closest_distance > (1 - similarity_threshold): # Adapter la condition pour L2 distance
            for k, v in pages_new.items():
                if chunk_new in v or v in chunk_new:
                    newpage = k
            for k, v in pages_old.items():
                if chunks_old[closest_chunk_old_index] in v or v in chunks_old[closest_chunk_old_index]:
                    oldpage = k
            differences.append(f"Ajout/Modification probable dans nouveau doc à la page {newpage}: \n'{chunk_new}'\n---------------\n(Distance au plus proche de l'ancien doc: {closest_distance:.4f})\nChunk comparé de l'ancien doc a la page {oldpage}:\n---------------\n'{chunks_old[closest_chunk_old_index]}")
        else:
            # Pour des modifications subtiles, vous pouvez aussi comparer le chunk original avec le chunk trouvé
            # si la similarité est juste au-dessus du seuil mais pas parfaite.
            # Cela nécessiterait une étape supplémentaire d'analyse.
            pass # Ici, le chunk est considéré comme similaire
    
    """for i, chunk_old in enumerate(chunks_old):
        query_embedding = embeddings_old[i].reshape(1, -1)
        distances, indices = index_new.search(query_embedding, k=1) 
        closest_distance = distances[0][0]
        closest_chunk_new_index = indices[0][0]
        if closest_distance > (1 - similarity_threshold):
            for k, v in pages_old.items():
                if chunk_old in v or v in chunk_old:
                    oldpage = k
            for k, v in pages_new.items():
                if chunks_new[closest_chunk_new_index] in v or v in chunks_new[closest_chunk_new_index]:
                    newpage = k
            differences.append(f"Suppression probable de l'ancien doc à la page {oldpage}: \n'{chunk_old}'\n---------------\n(Distance au plus proche du nouveau doc: {closest_distance:.4f})\nChunk comparé du nouveau doc a la page {newpage}:\n---------------\n'{chunks_new[closest_chunk_new_index]}")
"""
    return differences

In [249]:
differences = find_differences(old_chunks, np_old_embeddings, old_index, old_pages, new_chunks, np_new_embeddings, new_index, new_pages)

print(f"{len(differences)} différences notables ont été trouvées !")
print("\n##################\n")

for difference in differences:
    print(difference)
    print("\n##################\n")

282 différences notables ont été trouvées !

##################

Ajout/Modification probable dans nouveau doc à la page 635: 
'XXI
Comité de rédaction 
de la 6e édition
Coordination de l’ouvrage
Deruelle Philippe, CHU de Montpellier.
Fauvet Raffaèle, CHU de Caen.
Cortet Marion, hôpital de la Croix-Rousse, Lyon.
Foulon Arthur, CHU d’Amiens.
Huissoud Cyril, hôpital de la Croix-Rousse, Lyon.
Simon Emmanuel, CHU de Dijon.
Cette page a été intentionnellement laissée vierge
X'
---------------
(Distance au plus proche de l'ancien doc: 0.2000)
Chunk comparé de l'ancien doc a la page 24:
---------------
'Comité de rédaction de la 5e édition
Coordination de l'ouvrage
Deruelle Philippe, hôpitaux universitaires de Strasbourg.
Graesslin Olivier, hôpital Maison-Blanche, CHU de Reims.
Huissoud Cyril, hôpital de la Croix-Rousse, Lyon.
Robin Geoffroy, CHU de Lille.
Bonneau Claire, Institut Curie, Paris.
Fauvet Raffaelle, CHU de Caen.
Auteurs
Babin Guillaume, Institut Bergonie, Bordeaux.
Bereby Kahane M

# Agent synthétisation différences

In [240]:
class Difference(BaseModel):
    type: Literal['ajout', 'suppression', 'modification']
    page_nouveau_doc: int
    page_ancien_doc: int
    description: str

class Differences(BaseModel):
    items: List[Difference]

In [243]:
safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
]

differences_text = "\n##################\n".join(differences)
prompt = f"""
Voici une liste de différences potentielles détectées entre deux documents.
Veuillez analyser ces différences et fournir une liste claire et concise de chaque ajout ou modification significative,
en expliquant brièvement de quoi il s'agit.
Différences détectées :
{differences_text}

Liste des ajouts/modifications:
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        safety_settings=safety_settings,
        response_mime_type="application/json",
        response_schema=Differences
    )
).text

print(response)

{
  "items": [
    {
      "type": "modification",
      "page_nouveau_doc": 635,
      "page_ancien_doc": 24,
      "description": "Mise à jour de la liste du comité de rédaction de l'ouvrage, incluant de nouveaux membres et changements d'affiliation."
    },
    {
      "type": "ajout",
      "page_nouveau_doc": 29,
      "page_ancien_doc": 174,
      "description": "Ajout d'une abréviation DPA (Douleur pelvienne aiguë) et DPI"
    },
    {
      "type": "modification",
      "page_nouveau_doc": 29,
      "page_ancien_doc": 45,
      "description": "Modification de la section concernant le diagnostic prénatal, avec un focus sur les dispositions légales et les principes éthiques."
    },
    {
      "type": "modification",
      "page_nouveau_doc": 41,
      "page_ancien_doc": 45,
      "description": "Ajout d'une vignette clinique illustrant une interruption médicale de grossesse (IMG) pour raison de santé maternelle, ainsi que les considérations légales et éthiques associées."
    }

In [244]:
prompt = f"""
Voici une liste de toutes les différences remarquées entre 2 documents.
Vous devez transcrire toutes les informations fournies dans un format markdown esthétiquement clair, lisible, et précis.
Différences détectées :
{response}

Réponse:
"""

clean_response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(safety_settings=safety_settings)
).text

display(Markdown(clean_response))

## Différences entre les documents

Ce document résume les différences détectées entre deux versions d'un même document. Les informations sont classées par type de changement (ajout ou modification) et indiquent les pages concernées dans les deux versions.

**Légende:**

*   **Ancien Doc:** Numéro de page dans l'ancienne version du document.
*   **Nouveau Doc:** Numéro de page dans la nouvelle version du document.
*   **Description:** Description concise du changement effectué.

<br/>

### I. Modifications

| Ancien Doc | Nouveau Doc | Description                                                                                                                                                  |
| :----------: | :----------: | :------------------------------------------------------------------------------------------------------------------------------------------------------------ |
|      24      |     635      | Mise à jour de la liste du comité de rédaction de l'ouvrage, incluant de nouveaux membres et changements d'affiliation.                                   |
|      45      |      29      | Modification de la section concernant le diagnostic prénatal, avec un focus sur les dispositions légales et les principes éthiques.                         |
|      45      |      41      | Ajout d'une vignette clinique illustrant une interruption médicale de grossesse (IMG) pour raison de santé maternelle, ainsi que les considérations légales et éthiques associées. |
|      41      |      42      | Introduction d'une section sur l'éthique médicale, définissant les principes et les règles de conduite pour les professionnels de santé.                   |
|      41      |      42      | Ajout d'une liste de situations de départ et d'une hiérarchisation des connaissances concernant les violences sexuelles, incluant définitions, épidémiologie, et applications médicales de la législation.        |
|      54      |      58      | Explication détaillée des dispositions légales concernant le secret professionnel, le certificat médical, et le signalement judiciaire dans le cadre des violences sexuelles.                                |
|     816      |      68      | Présentation d'un cas clinique concernant une grossesse extra-utérine (GEU), incluant l'examen clinique, le diagnostic, et la prise en charge chirurgicale.                      |
|      73      |      68      | Informations complémentaires sur le diagnostic différentiel d'une grossesse extra-utérine et les traitements.                                            |
|      73      |      77      | Clarification des rubriques concernant les hémorragies génitales chez la femme.                                                                        |
|      73      |      78      | Définition des connaissances concernant les saignements utérins (ménorragies) et les situations d'urgence.                                                |
|      45      |      79      | Ajout d'un cas clinique concernant la prise en charge de méno-métrorragies abondantes liées à des fibromes utérins.                                         |
|      45      |      81      | Précisions sur les causes organiques et fonctionnelles des ménométrorragies, ainsi que sur le bilan de première intention.                                  |
|      81      |      81      | Ajout d'informations concernant l'évaluation du saignement, l'interrogatoire ciblé, et l'examen clinique pour orienter le diagnostic étiologique.          |
|      81      |      81      | Détails sur les examens biologiques et d'imagerie à réaliser devant une hémorragie génitale.                                                           |
|      85      |      83      | Précisions sur l'interprétation de la cytologie cervico-utérine et la conduite à tenir en cas de col pathologique.                                    |
|      85      |      83      | Informations supplémentaires concernant la prise en charge thérapeutique des hémorragies génitales, en fonction de l'étiologie et du désir de grossesse.   |
|      85      |      98      | Définition de la ménopause, de la ménopause anticipée, de l'insuffisance ovarienne prématurée, et de la périménopause.                                 |
|      85      |      98      | Informations concernant l'examen clinique à réaliser pour la ménopause.                                                                                 |
|     108      |     103      | Contre-indications au THM (traitement hormonal de la ménopause) et situations nécessitant une évaluation individuelle de la balance bénéfices-risques. |
|     108      |     103      | Situations de départ et hiérarchisation des connaissances concernant la contraception.                                                              |
|     381      |     103      | Informations sur les situations de départ.                                                                                                            |
|     131      |     119      | Bilan et suivi contraception: examens, surveillance, prise en charge.                                                                                     |
|     127      |     119      | Bilan clinique: examens, prescription d'une contraception hormonale.                                                                                      |
|     127      |     124      | Hiérarchisation des connaissances: prise en charge des grossesses, diagnostic de l'IVG,  etc.                                                             |
|     127      |     124      | Déroulement de la prise en charge: amélioration des patientes en demande d’IVG, loi de modernisation,  etc.                                                 |
|     312      |     129      | Traitements pour une morbidité des traitements.                                                                                                          |
|      73      |     129      | Traitements, causes cervicales, examens complémentaires.                                                                                                    |
|      81      |     129      | Traitements éthiques, et des protocoles de surveillance.                                                                                                 |
|      85      |     138      | Découverte: examen clinique , diagnostic de l'endométriose, éléments de l’interrogatoire etc..                                                            |
|     108      |     151      | Changement de l'âge de la patiente ainsi que de sa condition physique.                                                                                  |
|      41      |     151      | Déroulement d'un entrevue et législation pour signaler un enfant en danger.                                                                                |
|      81      |     152      | Examen de l'état émotif de la patiente, interrogatoire de l'agression récente à l'interrogatoire.                                                            |
|      81      |     154      | Ajout sur des traitements du volume uterus, le score de Higman et Béta-HCG.                                                                                 |
|      73      |     154      | Traitements des gènes chez la femme.                                                                                                                     |
|      73      |     159      | Changements des termes: les troubles du cycle menstruel et hiérarchisation des connaissances.                                                              |
|      73      |     160      | Les causes cervicales et les disposions des connaissances.                                                                                                  |
|      73      |     162      | Examen complémentaire  à ajouter aux connaissances                                                                                                         |
|      45      |     164      | Ajout d'information pertinente sur l' IMG.                                                                                                              |
|      41      |     165      | L' éthique et l'IMG souleve plusieurs questions.                                                                                                          |
|      41      |     170      | Référencement sur des connaissances additionnelles avec different point d'approche                                                                           |
|      45      |     178      | Traitement de certain point sans limite d'age.                                                                                                             |
|      45      |     180      | Diagnostic impliquant des questions de la fin de la grossess et en connaissant ces elements et ces facteurs.                                              |
|      45      |     181      | Traitement plus moderne et avec une explication plus approfondi du sujet.                                                                                   |
|      45      |     189      | Connaître et identifier les situations en cas de problèmes de grossess.                                                                                      |
|      45      |     192      | Information utile et pertinente sur des cas problématiques avec la grossess.                                                                                |
|      45      |     201      | Comment la moral des médecins peut avoir un impact sur les procédures des grossesse.                                                                           |
|      45      |     203      | L'importance de savoir résonner en face de complication en rapport avec la grossess.                                                                           |
|      45      |     204      | L'importance du cas par cas , l'adaptation du diagnostique et ces impacts.                                                                                  |
|      45      |     206      | Le suivies et les précautions que le praticien doit suivre en situation de complications.                                                                     |
|      45      |     213      | Certain trouble de la sexualité peuvent justifier des traitements bien spécifique.                                                                          |
|      45      |     214      | Hiérarchisation des connaissances et vignettes de diagnostic.                                                                                                |
|      45      |     217      | Il est essentiel de connaître les facteurs à prendre en compte avant de se pencher sur le cas d'un patient enceinte.                                          |
|      45      |     220      | Point d'approche pour le corps médical pour éviter un problème éthique ou immoral.                                                                              |
|      45      |     222      | Point d'approche en cas de saignement et des causes en résultant.                                                                                           |
|      45      |     226      | Information additionnelle pour améliorer le suivie médicale et pour informer le praticien en fonction du cas .                                            |
|      45      |     227      | La loi nous exige à nous conformer aux normes et au règlement en vigueur.                                                                                   |
|      45      |     229      | Mise à niveaux avec l'état actuelle des différentes formations reliées à la médecine.                                                                       |
|      45      |     231      | Changement qui permettent au praticien à mieux comprendre son approche aux problèmes de ses patient en cas de situations délicates.                          |
|      45      |     235      | Des exemples du quotidien avec les nouvelles recommendation.                                                                                                  |
|      45      |     238      | Information pour améliorer les soins apportées au patient en rapport avec le corps médicale.                                                                 |
|      45      |     241      | Il est important de bien comprendre la génétique humaine.                                                                                                   |
|      45      |     242      | Connaître les aspects de chaque diagnostiques et ces impacts en suivant l'état de la science.                                                                |
|      45      |     245      | La profession a changer et il est important de se réviser pour connaître ces subtilité.                                                                      |
|      45      |     246      | Il est essentielle de connaître tout nos droits et nos obligations en tant que professionnelle de la santé.                                                   |
|      45      |     249      | De nouveau et d'ancien approche sont à prendre en considération.                                                                                              |
|      45      |     250      | Toute les professionnelles de la santé doivent travailler en équipe pour assurer un suivie de grande qualité.                                                     |
|      45      |     252      | Il faut avoir l'esprit ouvert aux nouvelle pratiques et les nouveaux traitements et leurs suivies.                                                           |
|      45      |     257      | En cas de complications il faut pouvoir travailler sous pression et dans toute les situations.                                                                 |
|      45      |     264      | Le code de déontologie est là pour nous encadrer et nous supporter face aux problèmes éthique.                                                              |
|      45      |     266      | Les suivies et l'approche à prendre en compte sont essentielles pour une intervention de qualité.                                                            |
|      45      |     272      | On doit toujours continuer à améliorer ces connaissance et à les mettre à jour.                                                                              |
|      45      |     275      | Les examens complémentaires et leurs mises à jour.                                                                                                           |
|      45      |     278      | L'approfondissement de ces connaissance sont la clef du succès de nos compétences.                                                                             |
|      45      |     282      | La recherche pour être à niveaux de ces fonctions.                                                                                                            |
|      45      |     283      | Les techniques et connaissances de l'information en rapport avec les grossesses.                                                                                |
|      45      |     287      | Maitriser l'information à tous les niveaux.                                                                                                                  |
|      45      |     303      | Il faut savoir s'adapter avec toute les situation d'éthiques et de morale.                                                                                   |
|      45      |     304      | Les aspects de la population sont des fait à considérer .                                                                                                     |
|      45      |     314      | Les aspects de la grossesse sont des réalités à prendre en compte.                                                                                            |
|      45      |     318      | L'adaptation est essentielle pour assurer un suivi.                                                                                                           |
|      45      |     321      | On doit être conscient de toutes la réalité des cas et avoir des solutions en conséquence.                                                                     |
|      45      |     334      | Le déroulement et le pas à pas de toute ces pratiques.                                                                                                       |
|      45      |     337      | Tout les facteurs des risques à prendre en comptes avec cette intervention.                                                                                    |
|      45      |     341      | Il faut avoir les capacité pour évaluer chaque cas et apporter une solution en toute circonstances.                                                           |
|      45      |     356      | L'aspect sociale est un point essentiel dans les pratiques.                                                                                                    |
|      45      |     359      | Les aspect de tout ces éléments doivent être pris en compte avant même les situations qui implique l'éthique.                                                  |
|      45      |     361      | L'interrogation du sujet, l'objectivité et les questions à poser sont très importantes à connaître pour améliorer ces pratiques.                             |
|      45      |     366      | Connaître la situation du patient et ces effets de cause à effet.                                                                                             |
|      45      |     373      | Le respect est primordiale à tous les moments des pratiques.                                                                                                  |
|      45      |     375      | Il est important de connaitre les situation de départs.                                                                                                     |
|      45      |     381      | L'aspect psychologique du diagnostic et du traitement.                                                                                                       |
|      45      |     385      | Comment adapter son suivi en fonction du bilan des patient.                                                                                                   |
|      45      |     393      | La qualité et la rapidité des actions pour bien intervenir.                                                                                                     |
|      45      |     396      | Les protocoles à implémenter et les façon d'adopter une position et une réaction en cas de complications.                                                     |
|      45      |     397      | Il faut s'appuyer sur les connaissances pour pouvoir bien intervenir et réagir en conséquence.                                                              |
|      45      |     402      | Les aspect psychologique du patient sont aussi important que le diagnostique physique pour un suivis efficace.                                                 |
|      45      |     405      | L'examen à réalisé et le déroulement d'un entrevue en cas de problème.                                                                                        |
|      45      |     414      | La surveillance et les examen à privilégiées.                                                                                                                 |
|      45      |     417      | Les point de moral à respecter en tant que professionnelles et les conditions à respecter en cas de problèmes.                                               |
|      45      |     426      | Les outils technologiques mis à notre dispositions pour faciliter les interventions.                                                                           |
|      45      |     436      | Les examens pour bien déceler les pathologie et les solutions à y apporter.                                                                                 |
|      45      |     442      | Le diagnostique et les solutions à mettre en oeuvre.                                                                                                          |
|      45      |     446      | Comment bien intervenir du debut à la fin des procédures.                                                                                                       |
|      45      |     458      | C'est un récapitulatif de ce qui a été vue en dessus.                                                                                                           |
|      45      |     463      | L'aspect des infections pour assurer un suivis plus précis.                                                                                                     |
|      45      |     470      | La prise de médicament adéquat et approprié.                                                                                                                 |
|      45      |     475      | Le protocole à suivre étape par étape.                                                                                                                        |
|      45      |     482      | Il est important de connaitre tout ces protocoles à l'avance pour éviter des complications.                                                                      |
|      45      |     491      | Les facteurs de risques à connaitre en rapport avec les grossesse.                                                                                             |
|      45      |     494      | Recherche d'aspect techniques aux procédures.                                                                                                                  |
|      45      |     495      | Il est important de bien suivre les recommandations pour bien intervenir.                                                                                      |
|      45      |     507      | Aspect de toute la procédure et protocole.                                                                                                                    |
|      45      |     512      | Il faut mettre le confort et le bien être du patient en priorité.                                                                                             |
|      45      |     529      | Améliorer ces connaissances et ces formations sur une base continue.                                                                                           |
|      45      |     532      | Les aspects de la population en rapport avec des diagnostique et les façons d'y faire face.                                                                     |
|      45      |     534      | En tant que professionelles, on doit connaitre les avantages et les inconvénients de toutes situation.                                                         |
|      45      |     535      | Recherche et identifier les différentes pathologie pour bien intervenir.                                                                                       |
|      45      |     537      | Connaître et différencier les causes des diagnostiques en cas de fausses diagnostique.                                                                        |
|      45      |     538      | Connaître ce que l'ont doit s'attendre en fonction de chaque diagnostique .                                                                                    |
|      45      |     541      | Pour être à la hauteur de la situation , il faut bien se préparer et se poser les bonnes questions.                                                               |

<br/>

### II. Ajouts

| Ancien Doc | Nouveau Doc | Description                                                                                                                                                                                                                 |
| :----------: | :----------: | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
|     174      |      29      | Ajout d'une abréviation DPA (Douleur pelvienne aiguë) et DPI                                                                                                                                                       |
|      62      |      51      | Présentation d'un cas clinique concernant la prise en charge d'une victime de violences sexuelles, incluant l'examen clinique, les examens complémentaires, et le traitement.                                           |
|     312      |      67      | Ajout d'une section concernant les traitements.                                                                                                                        |
|      85      |      98      | Ajout d'une vignette clinique concernant la prise en charge de symptômes liés à la péri-ménopause.                                                                      |

<br/>

**Note:** Cette transcription met en évidence les principales modifications et ajouts entre les deux documents, fournissant une vue d'ensemble des changements effectués.  Les descriptions sont basées sur l'information fournie et sont conçues pour être aussi précises et informatives que possible.


In [245]:
with open("RESULT.md", 'w', encoding='utf-8') as f:
    f.write(clean_response)